# RSA密码破译

`Import the necesssary module`

In [23]:
import gmpy2
import owiener
import pollardrho as pr

`Load data`

In [24]:
def load_data(datanum):
    """
    Load RSA data to attack. 
    Args:
        datanum (int): The serial number of the RSA data
    Returns:
        n,e,c
    """
    with open(file="./test/data%d" % (datanum), mode='r') as file:
        n, e, c = file.read(256), file.read(256), file.read(256)
        n, e, c = int(n, 16), int(e, 16), int(c, 16)
        return n, e, c


def decode(s: str):
    s = s[-16:]
    out = ""
    for i in range(8):
        tmp = s[i * 2:(i + 1) * 2]
        out += chr(int(tmp, 16))
    print(out)

`Attack RSA`

In [25]:
def attack_broadcast(e, lst_c, lst_n):
    """
    Broadcast attack for small e
    """
    M = 1
    for n in lst_n:
        M = M * n
    lst_M = []
    for n in lst_n:
        lst_M += [M // n]
    m_e = M
    for i in range(len(lst_c)):
        m_e += lst_c[i] * lst_M[i] * gmpy2.invert(lst_M[i], lst_n[i])
    m_e = m_e % M
    m = gmpy2.iroot(m_e, e)
    i = 0
    while not m[1]:
        m_e += M
        i += 1
        m = gmpy2.iroot(m_e, e)
    return m

In [26]:
def Wiener_attack(n, e, c):
    """
    Wiener attack, for low decryption.
    """
    d = owiener.attack(e, n)
    if d != None:
        m = gmpy2.powmod(c, d, n)
        return m


for i in range(21):
    n, e, c = load_data(i)
    if Wiener_attack(n, e, c):
        print(i, "", hex(Wiener_attack(n, e, c)))


In [28]:
lst_n, lst_c, lst_i = [], [], []
for i in range(21):
    n, e, c = load_data(i)
    if e == 5:
        lst_n += [n]
        lst_c += [c]
        lst_i += [i]
print(lst_i, ":")
m = hex(attack_broadcast(5, lst_c, lst_n)[0])
print(m)
decode(m)


[4, 5, 7, 13, 18] :
0x9876543210abcdef0000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000007420697320612066
t is a f


In [33]:
def fmt(n):
    """
    Format decomposition of n
    """
    a = gmpy2.iroot(n, 2)
    a = a[0] + 1
    for _ in range(100000000):
        m = gmpy2.iroot(a * a - n, 2)
        if m[1]:
            return a - m[0], a + m[0]
        a = a + 1
    return 0, 0


def decrypt(p, q, e, c):
    """
    Given n = p * q, and e, c, decrypt the message. 
    """
    d = gmpy2.invert(e, (p - 1) * (q - 1))
    m = gmpy2.powmod(c, d, p * q)
    print(hex(m))
    decode(hex(m))


for i in range(21):
    n, e, c = load_data(i)
    p, q = fmt(n)
    if p:
        print(i, ":")
        decrypt(p, q, e, c)

8 :
0x9876543210abcdef00000008000000000000000000000000000000000000000000000000000000000000000000000000000000000000000077696c6c20676574
will get
10 :
0x9876543210abcdef00000009000000000000000000000000000000000000000000000000000000000000000000000000000000000000000020796f752066726f
 you fro


In [34]:
def attack_gcd(i, j):
    """
    Try to check if the n in i-th data and n in j-th data have common factor.
    """
    n1, e1, c1 = load_data(i)
    n2, e2, c2 = load_data(j)
    p1 = gmpy2.gcd(n1, n2)
    q1 = n1 // p1
    q2 = n2 // p1
    print(i, ":")
    decrypt(p1, q1, e1, c1)
    print(j, ":")
    decrypt(p1, q2, e2, c2)


for i in range(21):
    for j in range(21):
        n1, e1, c1 = load_data(i)
        n2, e2, c2 = load_data(j)
        if gmpy2.gcd(n1, n2) != 1 and i < j and n1 != n2:
            attack_gcd(i, j)


2 :
0x9876543210abcdef0000000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000006d204120746f2042
m A to B
19 :
0x9876543210abcdef0000000b00000000000000000000000000000000000000000000000000000000000000000000000000000000000000002e20496d6167696e
. Imagin


In [36]:
def attack_mod(e1, e2, c1, c2, n):
    """
    Attack when the same message was encrypted with the same n but different e.
    """
    e1, e2, c1, c2, n = int(e1), int(e2), int(c1), int(c2), int(n)
    s = gmpy2.gcdext(e1, e2)
    s1 = s[1]
    s2 = s[2]
    if s1 < 0:
        s1 = -s1
        c1 = gmpy2.invert(c1, n)
    elif s2 < 0:
        s2 = -s2
        c2 = gmpy2.invert(c2, n)
    m = (pow(c1, s1, n) * pow(c2, s2, n)) % n
    return hex(m)


for i in range(21):
    for j in range(21):
        n1, e1, c1 = load_data(i)
        n2, e2, c2 = load_data(j)
        if i < j and n1 == n2:
            print([i, j], ":")
            m=attack_mod(e1, e2, c1, c2, n1)
            print(m)
            decode(m)
            


[11, 14] :
0x9876543210abcdef0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000004d79207365637265
My secre


In [37]:
"""
Given the high bits of m, and a low encryption, use Copper-Smith attack. 
The detailed code is in Copper_Smith.sage.
"""
m_0 = 0x9876543210abcdef00000003000000000000000000000000000000000000000000000000000000000000000000000000000000000000000079696e67206f6620
m_6 = 0x9876543210abcdef000000040000000000000000000000000000000000000000000000000000000000000000000000000000000000000000416c626572742045
m_17 = 0x9876543210abcdef000000020000000000000000000000000000000000000000000000000000000000000000000000000000000000000000616d6f7573207361
print("0 :", hex(m_0))
decode(hex(m_0))
print("6 :", hex(m_6))
decode(hex(m_6))
print("17 :", hex(m_17))
decode(hex(m_17))


0 : 0x9876543210abcdef00000003000000000000000000000000000000000000000000000000000000000000000000000000000000000000000079696e67206f6620
ying of 
6 : 0x9876543210abcdef000000040000000000000000000000000000000000000000000000000000000000000000000000000000000000000000416c626572742045
Albert E
17 : 0x9876543210abcdef000000020000000000000000000000000000000000000000000000000000000000000000000000000000000000000000616d6f7573207361
amous sa


In [39]:
B_factorial = 1
for i in range(1, 200000):
    B_factorial *= i

In [40]:
def Pollard_p_1(n, e, c, i):
    """
    Pollard p-1 attack
    """
    a = gmpy2.powmod(2, B_factorial, n)
    p = gmpy2.gcd(n, a - 1)
    if p != 1:
        print(i, ":")
        decrypt(gmpy2.gcd(a - 1, n), n // gmpy2.gcd(a - 1, n), e, c)


for i in range(21):
    n, e, c = load_data(i)
    Pollard_p_1(n, e, c, i)


1 :
0x9876543210abcdef0000000600000000000000000000000000000000000000000000000000000000000000000000000000000000000000002054686174206973
 That is
3 :
0x9876543210abcdef000000050000000000000000000000000000000000000000000000000000000000000000000000000000000000000000696e737465696e2e
instein.
12 :
0x9876543210abcdef00000007000000000000000000000000000000000000000000000000000000000000000000000000000000000000000020224c6f67696320
 "Logic 


## Only data 9,15,16,20 are not attacked.

In [41]:
for i in [9, 15, 16, 20]:
    n, e, c = load_data(i)
    print(n, e, c, sep="\n")
    Wiener_attack(n, e, c)


99193711547257063160816850544214924340574358752670644615293764532335872088470223740970673347993652626497557387222167784182876395436088845281840169701654629849214222297784511349059698963212947299142320497759258889425182705042123217476724761095690092179821753840224757786599021225709340258545979566824267620959
65537
26054677793581772924866273737009673285775062802734786532404396138990264566536537921648515854012553861999940229349708989519156563830916553754762208466745321226835312974971739761769324569315525872096987367001543758380071859429619580182411498650200401467760546057912183435480924905200466941116258838789328064564
111178307033150739104608647474199786251516913698936331430121060587893564405482896814045419370401816305592149685291034839621072343496556225594365571727260237484885924615887468053644519779081871778996851601207571981072261232384577126377714005550318990486619636734701266032569413421915520143377137845245405768733
65537
139522218733405583349843513600726957213852511314574488296953